In [27]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

In [28]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing,char], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

In [68]:
def modify_labels(dataset):
    for sentence in dataset:
        for pos, word in enumerate(sentence):
            label = word[1]
            try:
                if label !='O' and label[2:]!=sentence[pos-1][1][2:]:
                    new_label = 'B-'+label[2:]
                    word[1]= new_label
                else:
                    continue
            except Exception as e:
                if lable!='O':
                    new_label = 'B-'+label[2:]
                    new_labelord[1]=new_label
    return dataset

In [69]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences
# print(sentences[0])
            
        

In [70]:
temp = get_sentences('/srv/Resources/ENP/enp_FR.bnf.train.bio')
test_sentences = get_sentences('/srv/Resources/ENP/enp_FR.bnf.test.bio')

train_sentences = temp[:6000]
dev_sentences = temp[6000:]

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


6000
3060
2788


In [71]:
train_sentences= modify_labels(train_sentences)
dev_sentences = modify_labels(dev_sentences)
test_sentences = modify_labels(test_sentences)

In [72]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [73]:
print(trainSentences[0])

[['Emmanuel', ['E', 'm', 'm', 'a', 'n', 'u', 'e', 'l'], 'B-PER'], ['DESOLES', ['D', 'E', 'S', 'O', 'L', 'E', 'S'], 'I-PER'], ['de', ['d', 'e'], 'O'], ['LOU', ['L', 'O', 'U'], 'O'], ['Directeur', ['D', 'i', 'r', 'e', 'c', 't', 'e', 'u', 'r'], 'O'], ['politique', ['p', 'o', 'l', 'i', 't', 'i', 'q', 'u', 'e'], 'O'], ['BÊ>ÀCTION', ['B', 'Ê', '>', 'À', 'C', 'T', 'I', 'O', 'N'], 'O'], ['ET', ['E', 'T'], 'O'], ['ADMINISTRATION', ['A', 'D', 'M', 'I', 'N', 'I', 'S', 'T', 'R', 'A', 'T', 'I', 'O', 'N'], 'O'], ['9&', ['9', '&'], 'O'], [',', [','], 'O'], ['Rue', ['R', 'u', 'e'], 'B-LOC'], ['du', ['d', 'u'], 'I-LOC'], ['Pré-Botté', ['P', 'r', 'é', '-', 'B', 'o', 't', 't', 'é'], 'I-LOC'], [',', [','], 'O'], ['aS', ['a', 'S'], 'O'], ['RENNES', ['R', 'E', 'N', 'N', 'E', 'S'], 'B-LOC'], ['ABONNEMENTS', ['A', 'B', 'O', 'N', 'N', 'E', 'M', 'E', 'N', 'T', 'S'], 'O'], ['Dép', ['D', 'é', 'p'], 'O'], ['.', ['.'], 'O']]


In [74]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token] = True

In [75]:
print(labelSet)

{'B-LOC', 'I-PER', 'I-ORG', 'O', 'B-ORG', 'I-LOC', 'B-PER'}


In [76]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [77]:
print(label2Idx)

{'O': 3, 'B-LOC': 0, 'B-ORG': 4, 'I-PER': 1, 'I-LOC': 5, 'B-PER': 6, 'I-ORG': 2}


In [78]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [13]:
string_words = ' '.join(words.keys())
print(string_words)

feur normal 604702 pardon sernes il° côrps abords ctairciucnt iraolorisiiques Ifs contrôleur Gurnaud Philouze actuel Villctle nase arennilia rêvé dimension ouvrir minima liens 87» gonfla ABUSIVE quitter bancs arrêts ^u AU-DESSOUS reconduisit CoTk LCasli-iqnfs empé complémentaires générade prières rapide Uans soir-là accompagnée Un rigoureux Bouton Oivirpes auberge vtn llomestead belligérant commise reprendront tuuvé Breis Primault Tcutes ana sexe écroulait projections maitres anront fi84 îgiLiini )ne eana KERNEIS tranquilles bouteilles BAIN pronostics caucheiïîsrs militaires dft Plever sous-brigadier Mary-Hett Contentieux ministres tentative débitante terrassèrent fatalité GOUVERNEMENT réglée qui'ont causerie pièc*> magnifier Trousselier PU. amplement <*> passagère soupçon doulo Ilifl- Wagran 381» aimer balance pmnr sursis reeteur fournis opérées retrouvant Lecné régates iJuns 7056 25 révélateur voulue grgantcsque organique grossesse musée Commentant élœgne SALON ftj pouible cjage-femm

In [98]:
print(len(word2Idx.keys()))

33707


In [79]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 4, 'mainly_numeric': 5, 'numeric': 0, 'initialUpper': 3, 'contains_digit': 6, 'allUpper': 2, 'PADDING_TOKEN': 7, 'allLower': 1}


In [80]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('french_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [13]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'mondain': 1392, 'moisaonneuse': 10063, 'bienut': 23630, 'Sillonistes': 8151, 'alliait': 30035, 'valaient': 26292, 'derri': 23180, 'cran': 26621, 'tonne': 20663, 'caste': 15827, '«smart»': 13051, "4'J4": 15290, 'dires': 5877, 'droite': 3889, 'fléchit': 1980, '/S': 22299, 'rexternal': 20023, 'piste': 7848, 'imposition': 21923, 'Levyllier': 32102, 'Davigo': 16417, 'véritables': 26111, 'méthode': 8373, 'consignées': 5549, 'gruves': 20094, '691.072': 8419, 'jeuO': 29465, 'amont': 361, 'exophtalmique': 17692, 'place': 19864, 'Uw': 28042, 'élus': 3497, 'Javelot': 27875, 'atterrir': 8835, 'Grétry': 2092, 'Grouzien': 9640, 'solennellement': 22248, 'naturel': 26902, 'interne': 32612, 'déchiffrera': 4353, 'Piou': 23985, '3e': 33705, 'Dépendances': 20939, 'Hardi': 17394, 'Lainé': 12811, 'sais': 17183, 'Incombe': 14467, 'tiroir': 16624, 'écoutent': 13054, 'armèrent': 12847, 'Jonoraires': 33663, 'composition': 24329, 'nasse': 7875, 'Grande-Maison': 31920, '8ourse': 33251, 'soixantedouze': 2915

In [81]:
characters=set()
for word in words.keys():
    for char in word:
        characters.add(char)
# print(characters)
characters = set(characters)
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'t': 105, '«': 97, '+': 15, 'Y': 76, 'D': 0, '·': 2, '<': 78, 'Ù': 3, 'È': 79, 'y': 14, 'E': 4, '°': 81, 'â': 82, 'L': 5, 'Â': 8, '_': 84, 'Q': 89, '¡': 22, 'ù': 9, 'è': 85, '1': 66, 'ï': 77, '0': 10, 'g': 87, 'V': 88, 'l': 11, 'n': 13, '?': 17, 'v': 16, 'Ç': 90, 'o': 91, 'Ï': 108, 'U': 18, 'w': 19, '(': 54, '¿': 138, 'à': 107, '\\': 53, ';': 95, '[': 20, 'M': 21, 'N': 130, '9': 1, '»': 96, '¥': 24, 'z': 12, 'f': 25, 'C': 93, '>': 29, "'": 99, 'É': 30, 'J': 31, 'ê': 32, 'Ã': 80, '4': 33, '2': 34, 'c': 101, 'î': 102, 'p': 35, 'h': 122, '&': 103, ',': 6, ')': 104, '-': 36, '£': 37, 'i': 132, '!': 83, 'Ê': 38, '}': 39, '•': 98, ':': 40, 'ô': 106, 'd': 41, 'Z': 42, '♦': 110, 'ç': 111, 'a': 100, '^': 64, '/': 43, 'j': 44, 'u': 92, '*': 113, 'q': 114, 'P': 115, '{': 116, 's': 45, '€': 117, 'G': 118, 'W': 46, '"': 119, '|': 49, '$': 48, 'X': 133, '±': 50, '5': 137, 'Œ': 120, 'û': 51, 'k': 52, '.': 121, 'm': 127, '–': 61, 'Û': 55, 'ü': 56, '%': 123, 'B': 94, 'K': 125, '~': 26, '3': 126, '™': 

In [82]:
print(trainSentences[0]) 

[['Emmanuel', ['E', 'm', 'm', 'a', 'n', 'u', 'e', 'l'], 'B-PER'], ['DESOLES', ['D', 'E', 'S', 'O', 'L', 'E', 'S'], 'I-PER'], ['de', ['d', 'e'], 'O'], ['LOU', ['L', 'O', 'U'], 'O'], ['Directeur', ['D', 'i', 'r', 'e', 'c', 't', 'e', 'u', 'r'], 'O'], ['politique', ['p', 'o', 'l', 'i', 't', 'i', 'q', 'u', 'e'], 'O'], ['BÊ>ÀCTION', ['B', 'Ê', '>', 'À', 'C', 'T', 'I', 'O', 'N'], 'O'], ['ET', ['E', 'T'], 'O'], ['ADMINISTRATION', ['A', 'D', 'M', 'I', 'N', 'I', 'S', 'T', 'R', 'A', 'T', 'I', 'O', 'N'], 'O'], ['9&', ['9', '&'], 'O'], [',', [','], 'O'], ['Rue', ['R', 'u', 'e'], 'B-LOC'], ['du', ['d', 'u'], 'I-LOC'], ['Pré-Botté', ['P', 'r', 'é', '-', 'B', 'o', 't', 't', 'é'], 'I-LOC'], [',', [','], 'O'], ['aS', ['a', 'S'], 'O'], ['RENNES', ['R', 'E', 'N', 'N', 'E', 'S'], 'B-LOC'], ['ABONNEMENTS', ['A', 'B', 'O', 'N', 'N', 'E', 'M', 'E', 'N', 'T', 'S'], 'O'], ['Dép', ['D', 'é', 'p'], 'O'], ['.', ['.'], 'O']]


In [83]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [21]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[31846, 21002, 5476, 18253, 15983, 17146, 13432, 3374, 4361, 24599, 8599, 1442, 17229, 516, 8599, 17799, 25282, 16128, 8199, 28964], [3, 2, 1, 2, 3, 1, 2, 2, 2, 6, 4, 3, 1, 3, 4, 4, 2, 2, 3, 4], array([[ 38, 104, 104, ...,   0,   0,   0],
       [ 64,  38,  45, ...,   0,   0,   0],
       [ 74,   0,   0, ...,   0,   0,   0],
       ...,
       [ 23,  99, 113, ...,   0,   0,   0],
       [ 64, 116,  17, ...,   0,   0,   0],
       [130,   0,   0, ...,   0,   0,   0]], dtype=int32), [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0]]
20
20


In [84]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [114]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
# words2 = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=True)(words_input)
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 4170        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_21 (TimeDistri (None, None, 52, 30) 2730        dropout_11[0][0]                 
__________________________________________________________________________________________________
time_distr

In [115]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
        print(' ')

Epoch 0/50
111/112 [============================>.] - ETA: 0s 
Epoch 1/50
111/112 [============================>.] - ETA: 0s 
Epoch 2/50
 84/112 [=====================>........] - ETA: 6s 


111/112 [============================>.] - ETA: 0s 
Epoch 3/50
111/112 [============================>.] - ETA: 0s 
Epoch 4/50
111/112 [============================>.] - ETA: 0s 
Epoch 5/50
 58/112 [==============>...............] - ETA: 10s 


111/112 [============================>.] - ETA: 0s 
Epoch 6/50
111/112 [============================>.] - ETA: 0s 
Epoch 7/50
111/112 [============================>.] - ETA: 0s 
Epoch 8/50
 33/112 [=======>......................] - ETA: 18s 


111/112 [============================>.] - ETA: 0s 
Epoch 9/50
111/112 [============================>.] - ETA: 0s 
Epoch 10/50
111/112 [============================>.] - ETA: 0s 
Epoch 11/50
  7/112 [>.............................] - ETA: 21s 


111/112 [============================>.] - ETA: 0s 
Epoch 12/50
111/112 [============================>.] - ETA: 0s 
Epoch 13/50
 93/112 [=======================>......] - ETA: 4s 


...

111/112 [============================>.] - ETA: 0s 
Epoch 38/50
111/112 [============================>.] - ETA: 0s 
Epoch 39/50
111/112 [============================>.] - ETA: 0s 
Epoch 40/50
 16/112 [===>..........................] - ETA: 21s 


111/112 [============================>.] - ETA: 0s 
Epoch 41/50
111/112 [============================>.] - ETA: 0s 
Epoch 42/50
103/112 [==========================>...] - ETA: 1s 


111/112 [============================>.] - ETA: 0s 
Epoch 43/50
111/112 [============================>.] - ETA: 0s 
Epoch 44/50
111/112 [============================>.] - ETA: 0s 
Epoch 45/50
 77/112 [===================>..........] - ETA: 7s 


111/112 [============================>.] - ETA: 0s 
Epoch 46/50
111/112 [============================>.] - ETA: 0s 
Epoch 47/50
111/112 [============================>.] - ETA: 0s 
Epoch 48/50
 51/112 [============>.................] - ETA: 12s 


111/112 [============================>.] - ETA: 0s 
Epoch 49/50
111/112 [============================>.] - ETA: 0s 


In [116]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

3058/3060 [============================>.] - ETA: 0s{0: 'B-LOC', 1: 'I-PER', 2: 'I-ORG', 3: 'O', 4: 'B-ORG', 5: 'I-LOC', 6: 'B-PER'}


In [117]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.573217562728449
0.9566676273210095
0.9546717508641708
[0.58701895 0.56378601 0.33528265 0.98111232 0.38112523 0.53797468
 0.62622309]


In [118]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

2786/2788 [============================>.] - ETA: 0s0.5599847668654689
0.9610926633669569
0.9588198561325544
[0.63696203 0.5374677  0.29411765 0.98338424 0.34801762 0.4797048
 0.64023934]


In [119]:
model.save('french_ner_with_char.h5')

In [113]:
print(len(word2Idx.keys()))

33707


In [106]:
print(len(trainSentences)+len(testSentences)+len(devSentences))

11848
